<a href="https://colab.research.google.com/github/Dyfox100/CUDA-Tutorials/blob/main/Basic_Operations_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### First Switch to GPU Instance

Double check to see if the CUDA compiler is installed and updated. The !(bang) operator in jupyter notebooks runs shell commands.

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


Installs the nvcc_plugin needed to run CUDA C/C++ from notebooks.

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-p2jusza2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-p2jusza2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=404da5f09ce403dd8c0e4916178172a0cd29b1ff15e307761e303df50e560c6d
  Stored in directory: /tmp/pip-ephem-wheel-cache-nxz9c7q5/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


Starts extension running in jupyter.

In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Simple program to make sure the C/C++ CUDA extension works. This won't run on gpu, but if the extension isn't working, colab will try to run this in python and it will blow up.

In [4]:
%%cu
#include <stdio.h>

/*just to check if the extension is working. None of his runs on the gpu.*/
int main() {
    printf("If this prints, the CUDA etension works!\n");
    return 0;
}


If this prints, the CUDA etension works!



# Terminology

### Host vs Device

Host -- The cpu that runs starts the kernel

Device -- GPUs or other computation devices that run the kernel.

Memory is normally listed as host or device memory.

### Grids and Blocks

Grid -- the entire space of threads in a kernel. Organized into blocks.

Block -- Threads are organized into blocks. Inside a block, threads have shared memory and can use synchronization operations. Outside a block, the kernel must be restarted to synchronize or communicate between different blocks.


In [5]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void hello_cuda() {
    printf("Hello from CUDA!\n");
}

int main() {
    // kernel launch params. First is num of blocks. Second is num threads in block.
    // Should print 6 times, 3 threads per block on 2 blocks.
    hello_cuda <<<2, 3>>>();
    
    // Waits until kernel completes. Necessary because main function will finish
    // before the kernel prints otherwise.
    cudaDeviceSynchronize();
    return 0;
}

Hello from CUDA!
Hello from CUDA!
Hello from CUDA!
Hello from CUDA!
Hello from CUDA!
Hello from CUDA!



### ID Numbers and Dim3 Coordinates

The variables threadIdx, blockIdx, blockDim, and gridDim can provide us with id numbers and dimensions for the gird, blocks, and threads. 

The grid and each block ar organized into a three dimensional coordinate system.
There is a struct in CUDA C/C++ that can be used to specify these three dimensional shapes. It's called dim3.

If using dim3 / (x, y, z) coordinates all coords must be > 0. 

There must be less than 1024 threads in x,y and 64 threads in z. And x * y * z must be less than 1024.

Must be less than 65536 thread blocks in y and z dirs and 2^32 - 1 in x .

Note that each thread runs independently, so the output is intermingled.

In [6]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void print_thread_id() {
    // Kernels have access to threadIDx structs that identify threads in a block.
    printf("Thread ID is: (%d, %d, %d)\n", threadIdx.x, threadIdx.y, threadIdx.z);
    // Also have access to blockIdx which identifys blocks in the grid.
    printf("Block ID is: (%d, %d, %d)\n", blockIdx.x, blockIdx.y, blockIdx.z);
    // blockDim structs hold the number of threads in each block. Same for all
    // blocks / threads.
    printf("Each block has %d by %d by %d blocks.\n",
           blockDim.x, blockDim.y, blockDim.z);
    // There is also a gridDim struct which gives dimensions of the grid (in number of blocks).
    printf("The grid has %d by %d by %d blocks.\n", 
           gridDim.x, gridDim.y, gridDim.z);
    // We can use this info to get the total number of threads.
    printf("The total number of threads is: %d.\n", 
           (blockDim.x * blockDim.y * blockDim.z) * (gridDim.x * gridDim.y * gridDim.z));
}

int main() {
    dim3 block(2, 1, 1);
    dim3 grid(2, 2, 1);
    
    print_thread_id <<<grid, block>>>();

    cudaDeviceSynchronize();
    return 0;
}

Thread ID is: (0, 0, 0)
Thread ID is: (1, 0, 0)
Thread ID is: (0, 0, 0)
Thread ID is: (1, 0, 0)
Thread ID is: (0, 0, 0)
Thread ID is: (1, 0, 0)
Thread ID is: (0, 0, 0)
Thread ID is: (1, 0, 0)
Block ID is: (1, 1, 0)
Block ID is: (1, 1, 0)
Block ID is: (0, 0, 0)
Block ID is: (0, 0, 0)
Block ID is: (1, 0, 0)
Block ID is: (1, 0, 0)
Block ID is: (0, 1, 0)
Block ID is: (0, 1, 0)
Each block has 2 by 1 by 1 blocks.
Each block has 2 by 1 by 1 blocks.
Each block has 2 by 1 by 1 blocks.
Each block has 2 by 1 by 1 blocks.
Each block has 2 by 1 by 1 blocks.
Each block has 2 by 1 by 1 blocks.
Each block has 2 by 1 by 1 blocks.
Each block has 2 by 1 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The grid has 2 by 2 by 1 blocks.
The total number of threads is: 8.
The total number of threads is: 8.
The total 

### Getting a unique thread index
To get a unique index for each thread:
1. Create a unique block index.
* Multiply the first dimension of the block id by the number of blocks in the other two dimensions.
* Add that quantity to the second block id dimension multiplied by the number of blocks in the third dimension.
* Add the third block id dimension to the previous quantity.


2. Create starting point for unique thread id.
* Multiply the unique block id by the numbers of threads in all dimensions.

3. Create a unique thread index.
* Multiply the thread id in the first dimension by the number of threads in the otehr two dims.
* Add that to the second dimension multiplied by the number of threads in the third.
* Add the third dimension to that quantity.

4. Add the quantiy in step 2 to step 3.


In [7]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void print_unique_index() {
    int uniqueBlock = blockIdx.x * gridDim.y * gridDim.z;
    uniqueBlock += blockIdx.y * gridDim.z;
    uniqueBlock += blockIdx.z;

    int blockOffset = uniqueBlock * blockDim.x * blockDim.y * blockDim.z;

    int thread = threadIdx.x * blockDim.y * blockDim.z;
    thread += threadIdx.y * blockDim.z;
    thread += threadIdx.z;

    int uniqueThreadID = blockOffset + thread;

    printf("%d\n", uniqueThreadID);
}



int main() {
    dim3 grid (2, 1, 1);
    dim3 block (2, 2, 3);
    print_unique_index <<<grid, block>>>();
    cudaDeviceSynchronize();
}

0
6
3
9
1
7
4
10
2
8
5
11
12
18
15
21
13
19
16
22
14
20
17
23



### Copying Memory to a Device and Back
1. Allocate memory on the host using a regular malloc.
2. Fill that memory with whatever you're trying to use on the gpu/device.
3. Allocate the same amount of memory on the device using cudaMalloc.
4. Use cudaMemcpy to copy memory from the host to the device.
5. Perform your computation on the device.
6. Use cudaMemcpy to copy the memory back to your host.
7. Free the memory on the device using cudaFree.


### Parameters
Paramters can be sent to the kernel by passing them in the function call.

They are passed by value.

This is a basic kernel that copies two integers to the device, passes pointers to those integers to the device kernel, adds them, and copies them back. This only runs on one thread, so it's not really taking advantage of the gpu. It's just a demonstration of how to copy things.

In [8]:
%%cu
#include <stdio.h>
#include <stdlib.h>


// Simple gpu function to add two variables.
__global__ void add(int *a, int *b, int *r){
    *r = *a + *b;
}

// Main function to run the gpu code.
int main() { 
    int a, b, r;

    // Device copies.
    int *d_a, *d_b, *d_r;

    // Allocates space on device for the three ints.
    // Puts pointers to this space in the variables d_a, d_b, d_r.
    cudaMalloc((void **)&d_a, sizeof(int));
    cudaMalloc((void **)&d_b, sizeof(int));
    cudaMalloc((void **)&d_r, sizeof(int));

   a = 2;
   b = 5;

    // Copy variables to device.
    cudaMemcpy(d_a, &a, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, &b, sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernel on the device.
    add<<<1,1>>>(d_a, d_b, d_r);

    // Copy the result back to the host and check for errors in copy.
    cudaError err = cudaMemcpy(&r, d_r, sizeof(int), cudaMemcpyDeviceToHost);

    // Check for errors. Err holds the error code, and cudaSuccess holds the expected code.
    if(err!=cudaSuccess) {
        printf("Error copying to Host: %s\n", cudaGetErrorString(err));
    }

    printf("Adding %d with %d on the gpu yields %d\n",a, b, r);

    // Need to free memory on the device.
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_r);

    return 0;

}

Adding 2 with 5 on the gpu yields 7



### Unified Memory Malloc API
CUDA also offers a simple api to allocate space on both the host and the device. Below is the add kernel with the host program changed to use the unified memory api.

In [9]:
%%cu
#include <stdio.h>
#include <stdlib.h>


// Simple gpu function to add two variables.
__global__ void add(int *a, int *b, int *r){
    *r = *a + *b;
}

// Main function to run the gpu code.
int main() { 
    int *a, *b,*r;

    // Allocates space on device and host for the three ints.
    cudaMallocManaged((void **)&a, sizeof(int));
    cudaMallocManaged((void **)&b, sizeof(int));
    cudaMallocManaged((void **)&r, sizeof(int));

   *a = 2;
   *b = 5;
   *r = 0;

    // Launch kernel on the device.
    add<<<1,1>>>(a, b, r);

    // Now we need to synchornize. The memcopy did this by default
    // in the last kernel since it waits for the current computation to finish.
    cudaDeviceSynchronize();

    printf("Adding %d with %d on the gpu yields %d\n", *a, *b, *r);

    // Need to free memory on the device.
    cudaFree(a);
    cudaFree(b);
    cudaFree(r);

    return 0;

}

Adding 2 with 5 on the gpu yields 7



### Querying Device Properties
CUDA provides an apit to query a devices properties. In paticular, this can be used to make sure we don't exceed any of the limits on the number of threads, or to check the warp size.

In [11]:
%%cu
#include <stdio.h>
 
// Print device properties
void printDevProp(cudaDeviceProp devProp)
{
    printf("Major number:                  %d\n",  devProp.major);
    printf("Minor number:                  %d\n",  devProp.minor);
    printf("Name:                          %s\n",  devProp.name);
    printf("Total global memory:           %u\n",  devProp.totalGlobalMem);
    printf("Total shared memory per block: %u\n",  devProp.sharedMemPerBlock);
    printf("Total registers per block:     %d\n",  devProp.regsPerBlock);
    printf("Warp size:                     %d\n",  devProp.warpSize);
    printf("Maximum memory pitch:          %u\n",  devProp.memPitch);
    printf("Maximum threads per block:     %d\n",  devProp.maxThreadsPerBlock);
    for (int i = 0; i < 3; ++i)
    printf("Maximum dimension %d of block:  %d\n", i, devProp.maxThreadsDim[i]);
    for (int i = 0; i < 3; ++i)
    printf("Maximum dimension %d of grid:   %d\n", i, devProp.maxGridSize[i]);
    printf("Clock rate:                    %d\n",  devProp.clockRate);
    printf("Total constant memory:         %u\n",  devProp.totalConstMem);
    printf("Texture alignment:             %u\n",  devProp.textureAlignment);
    printf("Concurrent copy and execution: %s\n",  (devProp.deviceOverlap ? "Yes" : "No"));
    printf("Number of multiprocessors:     %d\n",  devProp.multiProcessorCount);
    printf("Kernel execution timeout:      %s\n",  (devProp.kernelExecTimeoutEnabled ? "Yes" : "No"));
    return;
}
 
int main()
{
    // Number of CUDA devices
    int devCount;
    cudaGetDeviceCount(&devCount);
    printf("CUDA Device Query...\n");
    printf("There are %d CUDA devices.\n", devCount);
 
    // Iterate through devices
    for (int i = 0; i < devCount; ++i)
    {
        // Get device properties
        printf("\nCUDA Device #%d\n", i);
        cudaDeviceProp devProp;
        cudaGetDeviceProperties(&devProp, i);
        printDevProp(devProp);
    }
 
    return 0;
}

CUDA Device Query...
There are 1 CUDA devices.

CUDA Device #0
Major number:                  3
Minor number:                  7
Name:                          Tesla K80
Total global memory:           3407020032
Total shared memory per block: 49152
Total registers per block:     65536
Warp size:                     32
Maximum memory pitch:          2147483647
Maximum threads per block:     1024
Maximum dimension 0 of block:  1024
Maximum dimension 1 of block:  1024
Maximum dimension 2 of block:  64
Maximum dimension 0 of grid:   2147483647
Maximum dimension 1 of grid:   65535
Maximum dimension 2 of grid:   65535
Clock rate:                    823500
Total constant memory:         65536
Texture alignment:             512
Concurrent copy and execution: Yes
Number of multiprocessors:     13
Kernel execution timeout:      No



### Warps
A warp is a unit of threads that are executed simultaneously. Basically one of the streaming multi processors can execute the warp size of threads at a time. We may want to keep the thread block size a multiple of 32 so that we avoid wasting cycles at the end of each thread block.